# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [54]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [55]:
# Load csv file created in Part I
city_data_csv = "../VacationPy/city_data.csv"
# Create dataframe
city_data_df = pd.read_csv(city_data_csv)
city_data_df


,City_ID,City,Country,Date,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed
0,0,Busselton,AU,1666735447,-33.6500,115.3333,54.10,90,75,4.85
1,1,Mackay,AU,1666735548,-21.1500,149.2000,84.69,63,33,0.87
2,2,Ucluelet,CA,1666735548,48.9329,-125.5528,48.09,92,100,8.14
3,3,Kapaa,US,1666735548,22.0752,-159.3190,87.78,71,20,17.27
4,4,Provideniya,RU,1666735549,64.3833,-173.3000,28.51,64,100,15.99
...,...,...,...,...,...,...,...,...,...,...
567,567,Yamethin,MM,1666735741,20.4333,96.1500,72.01,94,18,4.00
568,568,Tiarei,PF,1666735742,-17.5333,-149.3333,82.42,84,100,24.36
569,569,Gorontalo,ID,1666735742,0.5412,123.0595,76.48,89,99,1.68
570,570,Yeppoon,AU,1666735743,-23.1333,150.7333,74.98,55,9,4.94


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [56]:
# Configure gmaps with API key
gmaps.configure(api_key = g_key)

In [59]:
# Convert Humidity to float and store
# HINT: be sure to handle NaN values
city_data_df = city_data_df.dropna()

# Store 'Lat' and 'Lng' into  locations 
locations = city_data_df[["Latitude", "Longitude"]]
humidity = city_data_df["Humidity"]

# Create a Humidity heatmap layer
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=100, 
                                 point_radius = 2)

fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [60]:
# Narrow down the DataFrame to find your ideal weather condition.
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temperature"] < 80) 
                                    & (city_data_df["Max Temperature"] > 70) 
                                    & (city_data_df["Wind Speed"] < 10) 
                                    & (city_data_df["Cloudiness"] == 0),:].dropna()
ideal_weather_df

,City_ID,City,Country,Date,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed
57,57,Stephenville,US,1666735565,32.2207,-98.2023,71.24,26,0,8.05
110,110,San Patricio,US,1666735588,28.0170,-97.5169,78.75,20,0,8.97
155,155,Kiryat Gat,IL,1666735601,31.6100,34.7642,77.68,40,0,6.31
161,161,Karumbākkam,IN,1666735603,12.6992,80.0933,76.78,71,0,4.65
170,170,Manoharpur,IN,1666735606,27.3000,75.9500,70.21,32,0,2.84
269,269,Nizwá,OM,1666735642,22.9333,57.5333,73.96,35,0,3.06
311,311,Saint-Pierre,RE,1666735214,-21.3393,55.4781,71.91,78,0,4.61
324,324,Mahon,ES,1666735656,39.8885,4.2658,70.36,88,0,6.91
353,353,Salalah,OM,1666735627,17.0151,54.0924,75.29,64,0,3.44
358,358,Kodīnar,IN,1666735670,20.7903,70.7031,74.98,62,0,8.39


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [61]:
hotel_df = ideal_weather_df[["City", "Country", "Latitude", "Longitude"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
57,Stephenville,US,32.2207,-98.2023,
110,San Patricio,US,28.0170,-97.5169,
155,Kiryat Gat,IL,31.6100,34.7642,
161,Karumbākkam,IN,12.6992,80.0933,
170,Manoharpur,IN,27.3000,75.9500,
269,Nizwá,OM,22.9333,57.5333,
311,Saint-Pierre,RE,-21.3393,55.4781,
324,Mahon,ES,39.8885,4.2658,
353,Salalah,OM,17.0151,54.0924,
358,Kodīnar,IN,20.7903,70.7031,


In [62]:
# Set parameters to serach for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through cities
for index, row in hotel_df.iterrows():
    # get lat, lng from dataframe
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while keeping original params in place
    params["location"] = f"{lat},{lng}"
    
    # Create base url to search nearby hotel     
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make request
    print(f"Retrieving results for city: {row['City']}")
    hotel_name = requests.get(base_url, params=params).json()
    
    # extract results
    hotel_name = hotel_name['results']
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name[0]['name']
        print(f"Closest hotel is {hotel_name[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = hotel_name[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

    
# Display hotel names on dataframe
hotel_df

Retrieving results for city: Stephenville
Missing field/result... skipping.
------------
Retrieving results for city: San Patricio
Missing field/result... skipping.
------------
Retrieving results for city: Kiryat Gat
Missing field/result... skipping.
------------
Retrieving results for city: Karumbākkam
Missing field/result... skipping.
------------
Retrieving results for city: Manoharpur
Missing field/result... skipping.
------------
Retrieving results for city: Nizwá
Missing field/result... skipping.
------------
Retrieving results for city: Saint-Pierre
Missing field/result... skipping.
------------
Retrieving results for city: Mahon
Missing field/result... skipping.
------------
Retrieving results for city: Salalah
Missing field/result... skipping.
------------
Retrieving results for city: Kodīnar
Missing field/result... skipping.
------------
Retrieving results for city: Sinnai
Missing field/result... skipping.
------------
Retrieving results for city: Ingham
Missing field/result

,City,Country,Latitude,Longitude,Hotel Name
57,Stephenville,US,32.2207,-98.2023,
110,San Patricio,US,28.0170,-97.5169,
155,Kiryat Gat,IL,31.6100,34.7642,
161,Karumbākkam,IN,12.6992,80.0933,
170,Manoharpur,IN,27.3000,75.9500,
269,Nizwá,OM,22.9333,57.5333,
311,Saint-Pierre,RE,-21.3393,55.4781,
324,Mahon,ES,39.8885,4.2658,
353,Salalah,OM,17.0151,54.0924,
358,Kodīnar,IN,20.7903,70.7031,


In [63]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [64]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))